In [1]:
import json
import re
import nltk
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import random

In [2]:
review_data = []
i = 0 
with open("reviews_Movies_and_TV.json", "r") as file:
    for line in file:
        i += 1 
        review = json.loads(line)
        review_data.append(review)
        if i == 200:
            break
        # process the review as needed
print(i)

with open("reduce.txt","w") as file:
    for word in review_data:
        file.write(word['reviewText'] + " ")

200


In [3]:
review_text = ''

In [4]:
for i in review_data:
    review_text = review_text + i['reviewText']

In [5]:
#Cleansing the Text
def Text_Cleaner(data):
    #To convert any sort of input data to a single string so it can be used in re expressions
    New_Data = str(data)
    #Converting the data to lowercase so there is no need to get case sencitive items seperately
    New_Data = New_Data.lower()
    #Removing Extra lines and "
    New_Data = New_Data.replace('\n', ' ')
    # Removing non-ASCII charachters from the data
    New_Data = re.sub(r'[^\x00-\x7F]+',' ', New_Data)
    # Any string starting with http and all the charachter till blank space would be replaced by <URL>
    New_Data = re.sub(r'http\S+', 'URL', New_Data)
    # Expecting the hashtag has no non-alpha numeric charachters else replace '\w' by ''\S'
    New_Data = re.sub(r'#\w+', 'HTAG', New_Data)
    # Expecting the mention has no non-alpha numeric charachters else replace '\w' by ''\S'
    New_Data = re.sub(r'@\w+', 'MTN', New_Data)
    # Replacing all the sets of '-' and '_' with a space
    New_Data = re.sub(r'-+', ' ', New_Data)
    New_Data = re.sub(r'_+', '', New_Data)
    # Replacing all the salutation '.' like 'Mr.' to 'Mr'
    New_Data = re.sub(r'mrs\.', 'mrs', New_Data)
    New_Data = re.sub(r'mr\.', 'mr', New_Data)
    # Replace subject's with subject_prop
    New_Data = re.sub(r'\'s', r' <Subjects_Prop> ', New_Data)
    # Replace {'re} with r + are
    New_Data = re.sub(r'\'re', r' are', New_Data)
    # Replace {'ll} with r + will
    New_Data = re.sub(r'\'ll', r' will', New_Data)
    # Replace {'m} with r + am
    New_Data = re.sub(r'\'m', r' am', New_Data)
    New_Data = re.sub(r'\\', '', New_Data)
    New_Data = re.sub(r"'", "", New_Data)
#   Replace Numbers
    New_Data = re.sub(r'\d+', '<Number>', New_Data)
#     Remove all punctions and symbols 
    New_Data = re.sub(r'[^\w\s]', '', New_Data)
    
    # Splitting string from punctuations
    #Puctuations after space
    New_Data = re.sub(r'(\w+)([.,!?:;\[\]*\"\'\(\)])', r'\1 \2', New_Data)
    #Puctuations before space
    New_Data = re.sub(r'([.,!?:;\[\]*\"\'\(\)])(\w+)', r'\1 \2', New_Data)
    
    # Remove extra spaces including line endings
    New_Data = re.sub(r'/s+',' ', New_Data)

    return New_Data  

In [6]:
cleansed_text = ''
cleansed_text = Text_Cleaner(review_text)

In [7]:
# nltk.download('stopwords')
# nltk.download('punkt')

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

In [9]:
# Load the stop words for English
stop_words = set(stopwords.words('english'))

# Tokenize the corpus (Words)
tokens = nltk.word_tokenize(cleansed_text)

# Remove stop words from the tokens
filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

# Join the filtered tokens to form a new sentence
new_corpus = ' '.join(filtered_tokens)
print(len(filtered_tokens))

6575


In [10]:
raw_text = new_corpus.split()
print(raw_text)

['great', 'tips', 'always', 'helping', 'complete', 'good', 'eats', 'collection', 'havent', 'tried', 'recipes', 'yet', 'soon', 'sometimes', 'Subjects_Prop', 'lovely', 'let', 'alton', 'entertain', 'usthis', 'great', 'pastry', 'guide', 'love', 'alton', 'Subjects_Prop', 'collections', 'break', 'baking', 'isnt', 'mystical', 'scary', 'might', 'even', 'try', 'recipes', 'dayi', 'admit', 'fan', 'giada', 'Subjects_Prop', 'cooking', 'great', 'expectations', 'ordered', 'set', 'however', 'crushed', 'still', 'love', 'giada', 'Subjects_Prop', 'cooking', 'set', 'way', 'food', 'network', 'make', 'money', 'really', 'cheated', 'dvd', 'Subjects_Prop', 'video', 'show', 'text', 'recipes', 'link', 'line', 'shows', 'computer', 'support', 'play', 'windows', 'media', 'player', 'set', 'contain', 'recipes', 'get', 'taping', 'shows', 'going', 'web', 'download', 'recipes', 'another', 'disappointment', 'transfer', 'quality', 'dvd', 'perhaps', 'ive', 'spoiled', 'hd', 'tivo', 'older', 'shows', 'ive', 'recorded', 'bett

In [11]:
torch.manual_seed()


embedding_dim = 350

def make_context_vector(context, word_t`o_idx):
    idxs = [word_to_idx[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

vocab = set(raw_text)
vocab_size = len(vocab)

word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}

data = []

for i in range(2, len(raw_text) - 2):
    context = [raw_text[i-2], raw_text[i-1],
               raw_text[i+1], raw_text[i+2]]
    target = raw_text[i]
    data.append((context, target))

In [12]:
class CBOW(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.proj = nn.Linear(embedding_dim, 300)
        self.output = nn.Linear(300, vocab_size)
        
    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1, -1)
        out = F.relu(self.proj(embeds))
        out = self.output(out)
        nll_prob = F.log_softmax(out, dim=-1)
        return nll_prob

model = CBOW(vocab_size, embedding_dim)
optimizer = optim.SGD(model.parameters(), lr=0.01)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
if use_cuda:
    model.cuda()
    losses = []
loss_function = nn.NLLLoss()

In [2]:
losses = []
loss_function = nn.NLLLoss()
num_neg_samples = 2

for epoch in range(10):
    total_loss = 0
    for context, target in data:
        
         # create a list of positive and negative samples
        pos_samples = [target]
        neg_samples = []
        while len(neg_samples) < num_neg_samples:
            # randomly sample a word from the vocabulary
            word = random.sample(vocab, 1)[0]
            # if the word is not the target word or one of the context words
            if word != target and word not in context:
                # add it to the negative samples list
                neg_samples.append(word)
                
        context_vector = make_context_vector(context, word_to_idx)
        # Remember PyTorch accumulates gradients; zero them out
        model.zero_grad()
        
         # calculate the positive sample loss
        nll_prob_pos = model(context_vector)
        loss_pos = loss_function(nll_prob_pos, Variable(torch.tensor([word_to_idx[target]])))
        
        # calculate the negative sample loss
        loss_neg = 0
        for neg_sample in neg_samples:
            nll_prob_neg = model(context_vector)
            loss_neg += loss_function(nll_prob_neg, Variable(torch.tensor([word_to_idx[neg_sample]])))
        
        # add up the posaitive and negative sample losses
        loss = loss_pos + loss_neg
        
        # backpropagation
        loss.backward()
        # update the parameters
        optimizer.step() 
        
        total_loss += loss.item()
    losses.append(total_loss)
    print(total_loss/len(data))

NameError: name 'nn' is not defined

In [14]:
context = ['process.','Computational','are', 'abstract']
# context_vector = make_context_vector(context, word_to_idx)
a = model(context_vector).data.numpy()
# print('Raw text: {}\n'.format(' '.join(raw_text)))
print('Test Context: {}\n'.format(context))
max_idx = np.argmax(a)
print('Prediction: {}'.format(idx_to_word[max_idx]))

Test Context: ['process.', 'Computational', 'are', 'abstract']

Prediction: christmas
